In [15]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import os
import datetime
# os.chdir("../")


In [16]:
def last_day_of_month(row):
    if row['month'] == 12:
        next_month = 1
        next_year = row['year'] + 1
    else:
        next_month = row['month'] + 1
        next_year = row['year']
    last_day = datetime.datetime(next_year, next_month, 1) - datetime.timedelta(days=1)
    return last_day

In [17]:
data=pd.read_csv("../merged/len3_prod_mon.csv",encoding="shift-jis",index_col=0)
# drop never treated group
count=1
data=data.dropna(subset=["release_year"])
data["r_date"]=pd.to_datetime(data["date"],format="%Y/%m/%d")
data["r_date"]
data["prod_date"]=data.apply(lambda row: last_day_of_month(row),axis=1)

In [18]:
data["elasped"]=data["prod_date"]-data["r_date"]
data["elasped"]=data["elasped"].dt.days//30
data["elasped"]

36       170
37       170
38       171
39       171
40       172
        ... 
39640    145
39753    260
39754    261
39755    262
39756    263
Name: elasped, Length: 16113, dtype: int64

1 observation が複数行にまたがっているので経過年数ダミー以外は落として1行にまとめよう

In [19]:
# data.loc[(data['elasped'] > 10) | (data["elasped"]< -5), 'elasped'] = np.nan
dummies=pd.get_dummies(data["elasped"],prefix="elasped")
dummies.columns = dummies.columns.str.replace('-', 'm').str.replace(r'\..*', '', regex=True)
colnames=dummies.columns.to_list()
if count:
    # count ver.
    aggregated_df = pd.concat([
        data.groupby(['year', 'id',"month"]).first(), 
        dummies.groupby([data['year'],data['id'],data["month"]]).sum()
    ], axis=1).reset_index()
else: 
    # dummy ver.
    aggregated_df = pd.concat([
        data.groupby(['year', 'id',"month"]).first(), 
        dummies.groupby([data['year'],data['id'],data["month"]]).apply(lambda x: (x[colnames] > 0).any().astype(int))
    ], axis=1).reset_index()
aggregated_df

,year,id,month,efficacy,rx_agg,rx_pro_agg,rx_pro_dom,otc_agg,otc_pro_agg,otc_pro_dom,...,elasped_395,elasped_396,elasped_397,elasped_398,elasped_399,elasped_400,elasped_401,elasped_402,elasped_403,elasped_404
0,2009,114,1,解熱鎮痛消炎剤,9749959,9528474,6804286,1328236,1328236,0,...,0,0,0,0,0,0,0,0,0,0
1,2009,114,2,解熱鎮痛消炎剤,9061579,8834341,7534695,2583588,2569825,13763,...,0,0,0,0,0,0,0,0,0,0
2,2009,114,3,解熱鎮痛消炎剤,11234470,11067590,8160160,2586928,2581213,5715,...,0,0,0,0,0,0,0,0,0,0
3,2009,114,4,解熱鎮痛消炎剤,12935859,12691332,10524766,2549867,2549642,225,...,0,0,0,0,0,0,0,0,0,0
4,2009,114,5,解熱鎮痛消炎剤,7769934,7731996,5017852,2500236,2500236,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4462,2023,629,4,その他の化学療法剤,"3,835,795","2,721,078","823,165",-,-,-,...,0,0,0,0,0,0,0,0,0,0
4463,2023,799,1,他に分類されない治療を主目的としない医薬品,"1,346,611","1,346,611","653,044","204,097","204,097","116,625",...,0,0,0,0,0,0,0,0,0,0
4464,2023,799,2,他に分類されない治療を主目的としない医薬品,"1,455,085","1,455,085","820,819","248,563","248,328","127,352",...,0,0,0,0,0,0,0,0,0,0
4465,2023,799,3,他に分類されない治療を主目的としない医薬品,"1,688,308","1,688,308","913,299","580,453","193,456",-,...,0,0,0,0,0,0,0,0,0,0


In [20]:
colnames

['elasped_m167',
 'elasped_m166',
 'elasped_m165',
 'elasped_m164',
 'elasped_m163',
 'elasped_m162',
 'elasped_m161',
 'elasped_m160',
 'elasped_m159',
 'elasped_m158',
 'elasped_m157',
 'elasped_m156',
 'elasped_m155',
 'elasped_m154',
 'elasped_m153',
 'elasped_m152',
 'elasped_m151',
 'elasped_m150',
 'elasped_m149',
 'elasped_m148',
 'elasped_m147',
 'elasped_m146',
 'elasped_m145',
 'elasped_m144',
 'elasped_m143',
 'elasped_m142',
 'elasped_m141',
 'elasped_m140',
 'elasped_m139',
 'elasped_m138',
 'elasped_m137',
 'elasped_m136',
 'elasped_m135',
 'elasped_m134',
 'elasped_m133',
 'elasped_m132',
 'elasped_m131',
 'elasped_m130',
 'elasped_m129',
 'elasped_m128',
 'elasped_m127',
 'elasped_m126',
 'elasped_m125',
 'elasped_m124',
 'elasped_m123',
 'elasped_m122',
 'elasped_m121',
 'elasped_m120',
 'elasped_m119',
 'elasped_m118',
 'elasped_m117',
 'elasped_m116',
 'elasped_m115',
 'elasped_m114',
 'elasped_m113',
 'elasped_m112',
 'elasped_m111',
 'elasped_m110',
 'elasped_m109

connect generic usage

In [21]:
dusage=pd.read_csv("../generic/generic_usage_imp.csv")
dusage
# generic from ndb
dgeneric=pd.read_csv("../merged/len3_ndb_generic_imputed.csv",index_col=0,encoding="shift-jis")
dgeneric

,薬効分類,year,generic_share_q,generic_share_r
0,114.0,2008.0,0.140940,0.049095
1,114.0,2009.0,0.153114,0.053230
2,114.0,2010.0,0.169573,0.058897
3,114.0,2011.0,0.187411,0.065125
4,114.0,2012.0,0.210339,0.073301
...,...,...,...,...
523,799.0,2019.0,0.006196,0.024777
524,799.0,2020.0,0.006056,0.022847
525,799.0,2021.0,0.007953,0.028184
526,799.0,2022.0,0.010886,0.047744


In [22]:
merged_df=aggregated_df.merge(dusage,left_on="year",right_on="Year")
merged_df=merged_df.merge(dgeneric,left_on=["id","year"],right_on=["薬効分類","year"])
merged_df

,year,id,month,efficacy,rx_agg,rx_pro_agg,rx_pro_dom,otc_agg,otc_pro_agg,otc_pro_dom,...,elasped_400,elasped_401,elasped_402,elasped_403,elasped_404,Year,Percentage,薬効分類,generic_share_q,generic_share_r
0,2009,114,1,解熱鎮痛消炎剤,9749959,9528474,6804286,1328236,1328236,0,...,0,0,0,0,0,2009,35.8,114.0,0.153114,0.053230
1,2009,114,2,解熱鎮痛消炎剤,9061579,8834341,7534695,2583588,2569825,13763,...,0,0,0,0,0,2009,35.8,114.0,0.153114,0.053230
2,2009,114,3,解熱鎮痛消炎剤,11234470,11067590,8160160,2586928,2581213,5715,...,0,0,0,0,0,2009,35.8,114.0,0.153114,0.053230
3,2009,114,4,解熱鎮痛消炎剤,12935859,12691332,10524766,2549867,2549642,225,...,0,0,0,0,0,2009,35.8,114.0,0.153114,0.053230
4,2009,114,5,解熱鎮痛消炎剤,7769934,7731996,5017852,2500236,2500236,0,...,0,0,0,0,0,2009,35.8,114.0,0.153114,0.053230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4194,2022,799,8,他に分類されない治療を主目的としない医薬品,1416628,1416628,748860,308849,308849,-,...,0,0,0,0,0,2022,79.0,799.0,0.010886,0.047744
4195,2022,799,9,他に分類されない治療を主目的としない医薬品,1899785,1598049,658356,391420,283334,-,...,0,0,0,0,0,2022,79.0,799.0,0.010886,0.047744
4196,2022,799,10,他に分類されない治療を主目的としない医薬品,1496351,1496351,655730,417711,335149,-,...,0,0,0,0,0,2022,79.0,799.0,0.010886,0.047744
4197,2022,799,11,他に分類されない治療を主目的としない医薬品,1325263,1325263,794279,209154,200543,-,...,0,0,0,0,0,2022,79.0,799.0,0.010886,0.047744


In [23]:
id_dummies=pd.get_dummies(merged_df["id"],prefix="id")
df_with_dummies=pd.concat([merged_df,id_dummies],axis=1)
df_with_dummies.rename(columns={"Percentage":"generic_per"},inplace=True)
df_with_dummies

,year,id,month,efficacy,rx_agg,rx_pro_agg,rx_pro_dom,otc_agg,otc_pro_agg,otc_pro_dom,...,id_264,id_265,id_332,id_339,id_399,id_441,id_449,id_625,id_629,id_799
0,2009,114,1,解熱鎮痛消炎剤,9749959,9528474,6804286,1328236,1328236,0,...,0,0,0,0,0,0,0,0,0,0
1,2009,114,2,解熱鎮痛消炎剤,9061579,8834341,7534695,2583588,2569825,13763,...,0,0,0,0,0,0,0,0,0,0
2,2009,114,3,解熱鎮痛消炎剤,11234470,11067590,8160160,2586928,2581213,5715,...,0,0,0,0,0,0,0,0,0,0
3,2009,114,4,解熱鎮痛消炎剤,12935859,12691332,10524766,2549867,2549642,225,...,0,0,0,0,0,0,0,0,0,0
4,2009,114,5,解熱鎮痛消炎剤,7769934,7731996,5017852,2500236,2500236,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4194,2022,799,8,他に分類されない治療を主目的としない医薬品,1416628,1416628,748860,308849,308849,-,...,0,0,0,0,0,0,0,0,0,1
4195,2022,799,9,他に分類されない治療を主目的としない医薬品,1899785,1598049,658356,391420,283334,-,...,0,0,0,0,0,0,0,0,0,1
4196,2022,799,10,他に分類されない治療を主目的としない医薬品,1496351,1496351,655730,417711,335149,-,...,0,0,0,0,0,0,0,0,0,1
4197,2022,799,11,他に分類されない治療を主目的としない医薬品,1325263,1325263,794279,209154,200543,-,...,0,0,0,0,0,0,0,0,0,1


In [24]:
# df_with_dummies.to_csv("len3_agg.csv",encoding="shift-jis")
if not(count):
    print("dummy")
    # dummy ver.
    df_with_dummies.to_csv("../merged/len3_prod_mon_agg_drop_never.csv",encoding="shift-jis")
else:    
    # count ver.
    df_with_dummies.to_csv("../merged/len3_prod_mon_sum_agg_drop_never.csv",encoding="shift-jis")